<a href="https://colab.research.google.com/github/LakshmanYES/My-School-Projects/blob/main/Sai_Lakshman_Ethakatla_Final_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Spark init
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("My Spark App").setMaster("local")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [ ]:
!wget --continue https://files.consumerfinance.gov/ccdb/complaints.csv.zip  -O /content/drive/MyDrive/data/complaints.csv.zip

--2022-05-06 02:14:41--  https://files.consumerfinance.gov/ccdb/complaints.csv.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 18.67.65.42, 18.67.65.67, 18.67.65.90, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|18.67.65.42|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
!unzip /content/drive/MyDrive/data/complaints.csv.zip -d /content/drive/MyDrive/data

Archive:  /content/drive/MyDrive/data/complaints.csv.zip
replace /content/drive/MyDrive/data/complaints.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: complaints_S
error:  invalid response [complaint]
replace /content/drive/MyDrive/data/complaints.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [s_S]
replace /content/drive/MyDrive/data/complaints.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/drive/MyDrive/data/complaints.csv  


In [ ]:
df_complaints_62 = spark.read.csv('/content/drive/MyDrive/data/complaints.csv', sep=',', header=True)

In [ ]:
df_complaints_62.show()

+-------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+--------------------------+-------------+--------------------+----------------------------+----------------+------------------+------------+
|Date received|             Product|         Sub-product|               Issue|           Sub-issue|Consumer complaint narrative|Company public response|             Company|State|ZIP code|         Tags|Consumer consent provided?|Submitted via|Date sent to company|Company response to consumer|Timely response?|Consumer disputed?|Complaint ID|
+-------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+--------------------------+-------------+--------------------+----------------------------+----------------+----

In [ ]:
df_complaints_63 = df_complaints_62.select(['Product','Consumer complaint narrative'])

In [ ]:
df_complaints_63.show()

+--------------------+----------------------------+
|             Product|Consumer complaint narrative|
+--------------------+----------------------------+
|Credit reporting,...|                        null|
|Credit reporting,...|                        null|
|        Student loan|                        null|
|Checking or savin...|                        null|
|Credit reporting,...|                        null|
|Credit reporting,...|                        null|
|     Debt collection|                        null|
|Checking or savin...|                        null|
|Credit reporting,...|                        null|
|Credit reporting,...|                        null|
|     Debt collection|                        null|
|Money transfer, v...|                        null|
|Credit reporting,...|                        null|
|Credit reporting,...|                        null|
|Credit reporting,...|                        null|
|Credit card or pr...|        It should be ille...|
|Credit repo

In [ ]:
df_complaints_64 = df_complaints_63.filter(df_complaints_63['Consumer complaint narrative'].isNotNull())

In [ ]:
df_complaints_64.show()

+--------------------+----------------------------+
|             Product|Consumer complaint narrative|
+--------------------+----------------------------+
|Credit card or pr...|        It should be ille...|
|Credit reporting,...|        I have been a Koh...|
|Checking or savin...|        Banking services ...|
| inoperable banki...|                       37381|
|Credit reporting,...|        In accordance wit...|
|Credit reporting,...|        hello dear agency...|
|     Debt collection|        XX/XX/2022. The d...|
|Credit reporting,...|        Notice to all, I,...|
| Experian Report ...|                          NV|
|Vehicle loan or l...|        Penfed Credit Uni...|
|Credit reporting,...|        On XX/XX/XXXX of ...|
|Credit reporting,...|        This is XXXX XXXX...|
|Credit reporting,...|        XXXX accordance w...|
|Company has respo...|               Servicemember|
|Credit reporting,...|        My name is XXXX X...|
|Credit reporting,...|        On XX/XX/2022 sen...|
|Credit card

In [ ]:
prod_list=["Mortgage","Student loan","Bank account or service"]
df_complaints_65=df_complaints_64.filter(df_complaints_64.Product.isin(prod_list))

In [ ]:
df_complaints_65.show()

+------------+----------------------------+
|     Product|Consumer complaint narrative|
+------------+----------------------------+
|    Mortgage|        My mother passed ...|
|    Mortgage|        Bank of America d...|
|    Mortgage|        "I have an equity...|
|    Mortgage|        I refinanced my h...|
|    Mortgage|        XXXX XXXX Mortgag...|
|    Mortgage|        Suntrust is sayin...|
|    Mortgage|        see attached emai...|
|    Mortgage|        XXXX OWNS SPECIAL...|
|    Mortgage|        My mortgage compa...|
|    Mortgage|        "The mortgage loa...|
|    Mortgage|        we have applied f...|
|    Mortgage|        During the Covid ...|
|    Mortgage|        I XXXX of XXXX I ...|
|Student loan|        I have been in pu...|
|    Mortgage|        On XX/XX/21, Hanc...|
|    Mortgage|        Contacted Vanderb...|
|    Mortgage|        LoanCare is my le...|
|Student loan|        FedLoans has been...|
|    Mortgage|        On or about mid X...|
|    Mortgage|        Mr. Copper

In [ ]:
(df_train, df_test) = df_complaints_65.randomSplit([0.8, 0.2], seed = 100)
print (df_train.first())

Row(Product='Bank account or service', Consumer complaint narrative='"ATM Skimming device placed on XXXX MA, XXXX between XXXX XXXX and XXXX XXXX. Bank statement outlined "" CASH WITHDRAWAL XXXX XXXX XXXX XXXX XXXX XXXX US . on XXXX XXXX at XXXX XXXX. \'\' Santander alerted me via snail mail that my debit card may have been a victim of fraud and it was disabled. After noticing charges being declined')


In [ ]:
print("The total no. of records in the training data:" ,df_train.count())

The total no. of records in the training data: 109096


In [ ]:
from pyspark.ml.feature import HashingTF, Tokenizer
import re
from pyspark.sql.functions import udf, length
from pyspark.sql.types import StringType, ArrayType
from pyspark.ml.feature import StopWordsRemover
filter = re.compile(r"^[a-zA-Z]+$")
match_udf = udf(lambda tokens: [token for token in tokens if filter.match(token)], ArrayType(StringType()))
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))

In [ ]:
def to_features(df):
  tokenizer = Tokenizer(inputCol="Consumer complaint narrative", outputCol="words")
  wordsData = tokenizer.transform(df)
  df_matched = wordsData.withColumn("words_matched", match_udf("words")). \
    select('Product', 'words_matched')
  # Remove stop words
  remover = StopWordsRemover(inputCol='words_matched', outputCol='words_clean')
  df_words_clean = remover.transform(df_matched). \
    select('Product', 'words_clean')
  # Stem text
  df_stemmed = df_words_clean.withColumn("words_stemmed", stemmer_udf("words_clean")). \
    select('Product', 'words_stemmed')

  # Generate vectors
  hashingTF = HashingTF(inputCol="words_stemmed", outputCol="rawFeatures") 
  featurizedData = hashingTF.transform(df_stemmed)
  return featurizedData

In [ ]:

df_train_features = to_features(df_train)
df_train_features.show()

+--------------------+--------------------+--------------------+
|             Product|       words_stemmed|         rawFeatures|
+--------------------+--------------------+--------------------+
|Bank account or s...|[skim, devic, pla...|(262144,[4226,659...|
|Bank account or s...|[citibank, pull, ...|(262144,[5078,211...|
|Bank account or s...|[america, cash, f...|(262144,[10529,21...|
|Bank account or s...|[inform, sent, us...|(262144,[1218,538...|
|Bank account or s...|[bank, close, bus...|(262144,[8473,253...|
|Bank account or s...|[wrong, close, ch...|(262144,[13970,25...|
|Bank account or s...|[charg, return, c...|(262144,[25262,25...|
|Bank account or s...|[heavili, adverti...|(262144,[892,5381...|
|Bank account or s...|[bank, xxxx, head...|(262144,[8538,153...|
|Bank account or s...|[xxxx, secur, cre...|(262144,[25363,33...|
|Bank account or s...|[updat, deposit, ...|(262144,[13970,25...|
|Bank account or s...|[open, new, check...|(262144,[25363,67...|
|Bank account or s...|[po

In [ ]:
df_products = df_test.select("Product").distinct()
df_products.show()

+--------------------+
|             Product|
+--------------------+
|            Mortgage|
|Bank account or s...|
|        Student loan|
+--------------------+



In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol = 'Product', outputCol = 'product_index')

In [ ]:
df_products_indexed = indexer.fit(df_products).transform(df_products)
df_products_indexed.show()

+--------------------+-------------+
|             Product|product_index|
+--------------------+-------------+
|            Mortgage|          1.0|
|Bank account or s...|          0.0|
|        Student loan|          2.0|
+--------------------+-------------+



In [ ]:
df_train_features_indexed = df_train_features.join(df_products_indexed, 'Product')
df_train_features_indexed.show()

+--------+--------------------+--------------------+-------------+
| Product|       words_stemmed|         rawFeatures|product_index|
+--------+--------------------+--------------------+-------------+
|Mortgage|     [payment, list]|(262144,[14376,21...|          1.0|
|Mortgage|[well, fargo, ref...|(262144,[32128,75...|          1.0|
|Mortgage|[xxxx, set, law, ...|(262144,[87595,18...|          1.0|
|Mortgage|[alreadi, post, t...|(262144,[892,5078...|          1.0|
|Mortgage|[new, inform, new...|(262144,[892,5078...|          1.0|
|Mortgage|[intent, defraud,...|(262144,[5116,612...|          1.0|
|Mortgage|[request, includ,...|(262144,[16474,36...|          1.0|
|Mortgage|[month, ago, exac...|(262144,[4075,421...|          1.0|
|Mortgage|[separ, complaint...|(262144,[5381,439...|          1.0|
|Mortgage|[understand, home...|(262144,[12650,12...|          1.0|
|Mortgage|[current, respons...|(262144,[126293,1...|          1.0|
|Mortgage|[year, good, stan...|(262144,[11395,16...|          

In [ ]:
from pyspark.ml.classification import NaiveBayes
   
nb = NaiveBayes(labelCol="product_index",\
    featuresCol="rawFeatures", smoothing=1.0,\
    modelType="multinomial")
model = nb.fit(df_train_features_indexed)

In [ ]:
df_train_features_indexed.show()

+--------+--------------------+--------------------+-------------+
| Product|       words_stemmed|         rawFeatures|product_index|
+--------+--------------------+--------------------+-------------+
|Mortgage|     [payment, list]|(262144,[14376,21...|          1.0|
|Mortgage|[well, fargo, ref...|(262144,[32128,75...|          1.0|
|Mortgage|[xxxx, set, law, ...|(262144,[87595,18...|          1.0|
|Mortgage|[alreadi, post, t...|(262144,[892,5078...|          1.0|
|Mortgage|[new, inform, new...|(262144,[892,5078...|          1.0|
|Mortgage|[intent, defraud,...|(262144,[5116,612...|          1.0|
|Mortgage|[request, includ,...|(262144,[16474,36...|          1.0|
|Mortgage|[month, ago, exac...|(262144,[4075,421...|          1.0|
|Mortgage|[separ, complaint...|(262144,[5381,439...|          1.0|
|Mortgage|[understand, home...|(262144,[12650,12...|          1.0|
|Mortgage|[current, respons...|(262144,[126293,1...|          1.0|
|Mortgage|[year, good, stan...|(262144,[11395,16...|          

In [ ]:
df_test_features = to_features(df_test)
df_test_features.show()

+--------------------+--------------------+--------------------+
|             Product|       words_stemmed|         rawFeatures|
+--------------------+--------------------+--------------------+
|Bank account or s...|[give, fund, dire...|(262144,[1218,377...|
|Bank account or s...|[bank, say, avail...|(262144,[13970,21...|
|Bank account or s...|[bank, bought, ch...|(262144,[5381,157...|
|Bank account or s...|[deposit, check, ...|(262144,[4214,139...|
|Bank account or s...|[save, checj, yes...|(262144,[1218,243...|
|Bank account or s...|[chase, custom, s...|(262144,[9177,126...|
|Bank account or s...|[open, pinnacl, c...|(262144,[25363,12...|
|Bank account or s...|[open, cap, one, ...|(262144,[5765,814...|
|Bank account or s...|[open, onlin, ban...|(262144,[892,1218...|
|Bank account or s...|[paid, xxxx, cred...|(262144,[4214,928...|
|Bank account or s...|[receiv, letter, ...|(262144,[13970,25...|
|Bank account or s...|[recent, open, xx...|(262144,[6122,139...|
|Bank account or s...|[xx

In [ ]:
df_test_features_indexed = df_test_features.join(df_products_indexed, 'Product')
df_test_features_indexed.show()

+--------+--------------------+--------------------+-------------+
| Product|       words_stemmed|         rawFeatures|product_index|
+--------+--------------------+--------------------+-------------+
|Mortgage|[capit, one, matu...|(262144,[4075,595...|          1.0|
|Mortgage|[parti, insur, cl...|(262144,[4214,178...|          1.0|
|Mortgage|[call, multipl, n...|(262144,[925,3176...|          1.0|
|Mortgage|[bank, america, h...|(262144,[16474,17...|          1.0|
|Mortgage|[settlement, give...|(262144,[21161,25...|          1.0|
|Mortgage|[charg, late, fee...|(262144,[4214,576...|          1.0|
|Mortgage|[xxxx, xxxx, tran...|(262144,[22686,34...|          1.0|
|Mortgage|[period, xxxx, xx...|(262144,[25042,31...|          1.0|
|Mortgage|[usda, fix, inter...|(262144,[4214,538...|          1.0|
|Mortgage|[write, today, we...|(262144,[5768,164...|          1.0|
|Mortgage|[well, fargo, spo...|(262144,[16004,21...|          1.0|
|Mortgage|[unfair, decept, ...|(262144,[1218,612...|          

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictions = model.transform(df_test_features_indexed)
predictions.select("Product", "product_index", 
    "probability", "prediction").show()
evaluator =\
    MulticlassClassificationEvaluator(labelCol="product_index",\
    predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+--------+-------------+--------------------+----------+
| Product|product_index|         probability|prediction|
+--------+-------------+--------------------+----------+
|Mortgage|          1.0|[1.01311316166707...|       1.0|
|Mortgage|          1.0|[1.70612602005219...|       1.0|
|Mortgage|          1.0|[3.58696121726528...|       1.0|
|Mortgage|          1.0|[0.64247281242875...|       0.0|
|Mortgage|          1.0|[2.45036124765516...|       1.0|
|Mortgage|          1.0|[1.37296402065919...|       1.0|
|Mortgage|          1.0|[3.60775443177137...|       1.0|
|Mortgage|          1.0|[1.72597573548195...|       1.0|
|Mortgage|          1.0|[1.32698669756330...|       1.0|
|Mortgage|          1.0|[6.09441457993384...|       1.0|
|Mortgage|          1.0|[1.58460546663756...|       1.0|
|Mortgage|          1.0|[8.75091351558510...|       1.0|
|Mortgage|          1.0|[2.08266655345479...|       1.0|
|Mortgage|          1.0|[4.28160416409425...|       1.0|
|Mortgage|          1.0|[1.4424